In [1]:
import json
from snowflake.snowpark.session import Session

# Reading Snowflake Connection Details
snowflake_connection_cfg = json.loads(open('snowflake_connection.json').read())

# Creating Snowpark Session
session = Session.builder.configs(snowflake_connection_cfg).create()

# Create a fresh & new schema
session.sql('CREATE OR REPLACE DATABASE PYCON_DEMO').collect()
session.sql('CREATE SCHEMA PYCON_DEMO._0_RAW').collect()
session.sql('CREATE SCHEMA PYCON_DEMO._1_ENRICHED').collect()
session.sql('CREATE SCHEMA PYCON_DEMO._2_MODELLED').collect()
session.use_schema('PYCON_DEMO._0_RAW')

In [2]:
# Function to generate Data Creation SQL
def generate_data(table_name, num_rows):
    generate_data_sql = """
    -- Create fake data
    create or replace table {} (email string, gender string, MEMBERSHIP_STATUS string,  MEMBERSHIP_LENGTH double, AVG_SESSION_LENGTH double, TIME_ON_APP double, TIME_ON_WEBSITE double, YEARLY_SPENT double) as 
    with profiles as (
    select concat(lower(randstr(10, random())), '@', lower(randstr(5, random())), '.com') as EMAIL,
           case when uniform(1,10,random())<=7 then 'MALE'
                when uniform(1,10,random())<=10 then 'FEMALE'
           end as GENDER,
           uniform(100,75000,random()) / 100 as YEARLY_SPENT,
           case when YEARLY_SPENT < 150 then 'BASIC'
                when YEARLY_SPENT < 250 then 'BRONZE'
                when YEARLY_SPENT < 350 then 'SILVER'
                when YEARLY_SPENT < 550 then 'GOLD'
                when YEARLY_SPENT < 650 then 'PLATIN'
                when YEARLY_SPENT < 720 then 'DIAMOND'
           end as MEMBERSHIP_STATUS,
           case when YEARLY_SPENT < 150 then null
                when YEARLY_SPENT < 250 then uniform(50,150,random()) / 100
                when YEARLY_SPENT < 350 then uniform(250,350,random()) / 100
                when YEARLY_SPENT < 550 then uniform(300,550,random()) / 100
                when YEARLY_SPENT < 650 then uniform(500,750,random()) / 100
                when YEARLY_SPENT < 720 then uniform(700,1000,random()) / 100
           end as MEMBERSHIP_LENGTH,
           case when YEARLY_SPENT < 120 then null
                when YEARLY_SPENT < 150 then uniform(500,750,random()) / 100
                when YEARLY_SPENT < 250 then uniform(700,1000,random()) / 100
                when YEARLY_SPENT < 350 then uniform(900,2000,random()) / 100
                when YEARLY_SPENT < 550 then uniform(1900,2700,random()) / 100
                when YEARLY_SPENT < 650 then uniform(2500,3200,random()) / 100
                when YEARLY_SPENT < 1000 then uniform(3000,4000,random()) / 100
           end as AVG_SESSION_LENGTH,
           case when YEARLY_SPENT < 150 then uniform(5000,7500,random()) / 100
                when YEARLY_SPENT < 250 then uniform(7300,10000,random()) / 100
                when YEARLY_SPENT < 350 then uniform(9500,20000,random()) / 100
                when YEARLY_SPENT < 370 then null
                when YEARLY_SPENT < 550 then uniform(19000,27000,random()) / 100
                when YEARLY_SPENT < 650 then uniform(25000,32000,random()) / 100
                when YEARLY_SPENT < 1000 then uniform(30000,40000,random()) / 100
           end as TIME_ON_APP,
           case when YEARLY_SPENT < 300 then uniform(5000,7500,random()) / 100
                when YEARLY_SPENT < 500 then uniform(7000,15000,random()) / 100
                when YEARLY_SPENT < 520 then null
                when YEARLY_SPENT < 1000 then uniform(14000,30000,random()) / 100
           end as TIME_ON_WEBSITE
    from table(generator(rowcount=>{})))
    select email, gender, MEMBERSHIP_STATUS, MEMBERSHIP_LENGTH, AVG_SESSION_LENGTH, TIME_ON_APP, TIME_ON_WEBSITE, YEARLY_SPENT from profiles;
    """.format(table_name, num_rows)
    return generate_data_sql

In [3]:
session.sql('ALTER WAREHOUSE COMPUTE_WH SET WAREHOUSE_SIZE=XXXLARGE').collect()

# Generating sample data
# 1 million rows
session.sql(generate_data('ECOMMERCE_CUSTOMERS_1M', 1000000)).collect()
# 100 million rows
session.sql(generate_data('ECOMMERCE_CUSTOMERS_100M', 100000000)).collect()

session.sql('ALTER WAREHOUSE COMPUTE_WH SET WAREHOUSE_SIZE=XSMALL').collect()

[Row(status='Statement executed successfully.')]